In [14]:
import pandas as pd

In [15]:
hero_id = pd.json_normalize(pd.read_json("../data/Hero ID.txt").at["heroes","result"])
hero_id["name"] = hero_id["name"].str.replace("npc_dota_hero_","")

In [16]:
hero_id.sort_values(by="name")

,name,id
100,abaddon,102
71,alchemist,73
67,ancient_apparition,68
0,antimage,1
110,arc_warden,113
...,...,...
20,windrunner,21
109,winter_wyvern,112
29,witch_doctor,30
40,wraith_king,42


In [17]:
ori = pd.read_csv("../data/raw_data_csv.csv")
print(ori.shape)
ori = ori.drop_duplicates()
print(ori.shape)

(12114, 14)
(9688, 14)


In [18]:
df = ori[["radiant_win","radiant_team","dire_team"]]

x = df["radiant_team"].str.split(r',', expand=True).stack().reset_index(level=1, drop=True).to_frame('radiant_team')
#x["radiant_team"] = x["radiant_team"].map(dict(zip(hero_id["id"].astype("str"),hero_id["name"])))
x = pd.get_dummies(x, prefix='', prefix_sep="", columns=['radiant_team']).groupby(level=0).sum().astype("int")
x.columns = x.columns.map(int)
x = x.reindex(sorted(x.columns), axis=1)

y = df["dire_team"].str.split(r',', expand=True).stack().reset_index(level=1, drop=True).to_frame('dire_team')
#y["dire_team"] = y["dire_team"].map(dict(zip(hero_id["id"].astype("str"),hero_id["name"])))
y = pd.get_dummies(y, prefix='', prefix_sep="", columns=['dire_team']).groupby(level=0).sum().astype("int")
y.columns = y.columns.map(int)
y = y.reindex(sorted(y.columns), axis=1)

hero = x-y
hero["radiant_win"] = df["radiant_win"].astype("int")

In [19]:
hero.to_csv("../data/preprocessed_data.csv", index=False)